In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import cv2
import IPython
from six.moves import urllib

print(tf.__version__)

In [2]:
# # Code for loading Data
# dataset = keras.preprocessing.image_dataset_from_directory(
#     directory="/var/trainingData/",
#     labels="inferred",
#     label_mode="int",
#     batch_size=32,
#     image_size=(180, 180),
#     shuffle=True,
#     seed=123,
# )

In [ ]:
# Assume your directory structure is:
# main_directory/
# ...class_a/
# ......a_image_1.jpg
# ......a_image_2.jpg
# ...class_b/
# ......b_image_1.jpg
# ......b_image_2.jpg

# Create a training dataset from the main directory with 70% of data
train_ds = tf.keras.utils.image_dataset_from_directory(
  directory="/var/trainingData/",
  labels="inferred",
  label_mode="int",
  image_size=(180, 180),
  batch_size=32,
  shuffle=False,
  seed=123,
  validation_split=0.95,
  subset="training",
)

# Create a validation dataset from the main directory with 30% of data
val_ds = tf.keras.utils.image_dataset_from_directory(
  directory="/var/trainingData/",
  labels="inferred",
  label_mode="int",
  image_size=(180, 180),
  batch_size=32,
  shuffle=False,
  seed=123,
  validation_split=0.03,
  subset="validation",
)

In [4]:
# for image in train_ds:
#     print(image)

In [ ]:
train_ds.map(lambda x, y: (tf.cast(x, tf.float64), y))

In [6]:
# import os
# import glob

# # Define the path to the directory
# directory_path = '/var/trainingData/'

# # Get a list of all subdirectories
# subdirectories = [x[0] for x in os.walk(directory_path)]

In [7]:
# # Initialize lists to store image file paths and labels
# image_paths = []
# labels = []

# # Loop over each subdirectory
# for i, subdir in enumerate(subdirectories[1:]):  # we skip the first element as it is the root directory
#     # Find all JPEG or JPG files in the current subdirectory
#     files = glob.glob(subdir + '/*.jpg') + glob.glob(subdir + '/*.jpeg')
    
#     # Add files and labels to the lists
#     image_paths += files
#     labels += [i] * len(files)

In [8]:
# import tensorflow as tf

# # Create a dataset of image file paths and labels
# dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))


In [9]:
# def load_and_preprocess_image(path, label):
#     # Load the image
#     image = tf.io.read_file(path)
#     image = tf.image.decode_jpeg(image, channels=3)
    
#     # Resize the image
#     image = tf.image.resize(image, [224, 224])
    
#     # Normalize pixel values
#     image /= 255.0
    
#     return image, label


In [10]:
# Apply the function to all elements in the dataset
# dataset = dataset.map(load_and_preprocess_image)


In [11]:
# # Batch, shuffle, and prefetch
# dataset = dataset.batch(32)
# dataset = dataset.shuffle(buffer_size=10000)
# dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


In [12]:
# import sys

In [13]:
# print(sys.getsizeof(train_ds))

In [14]:
# print(list(train_ds.as_numpy_iterator()))
# print(type(train_ds))

In [15]:
# import pandas as pd

# train_df = pd.DataFrame(train_ds)

In [16]:
# print(len(train_df))

In [17]:
# print(train_df.info())

In [18]:
# print(len(train_ds))

In [ ]:
# Get the class names from the train_ds
class_names = train_ds.class_names

# Print the class names
print(class_names)

# Get the number of labels
num_labels = len(class_names)

# Print the number of labels
print(num_labels)


In [20]:
# # Define a filter function that checks if the image has a non-zero shape
# def filter_zero_images(image, label):
#   # Get the shape of the image
#   image_shape = tf.shape(image)
#   # Check if the shape is non-zero
#   non_zero = tf.math.reduce_any(image_shape > 0)
#   # Return True or False
#   return non_zero

# # Apply the filter function to your batch dataset
# train_ds = train_ds.filter(filter_zero_images)

In [21]:
# # Get the number of elements in the dataset as a tensor
# num_elements = tf.data.experimental.cardinality(train_ds)

# # Convert the tensor to a Python integer
# num_elements = tf.get_static_value(num_elements)

# # Print the number of elements
# print(num_elements)


In [22]:
# # Get the batch size
# batch_size = 32

# # Get the number of images by multiplying the number of elements by the batch size
# num_images = num_elements * batch_size

# # Print the number of images
# print(num_images)


In [23]:
# import torchvision.transforms as transforms

# # Define a transform to convert PIL images to tensors
# transform = transforms.ToTensor()

# # Iterate through each sample in the train_dataset
# for sample in train_dataset:
#     # Get the image tensor from the sample
#     image_tensor = sample[0]

#     # Check if the image tensor is empty
#     if image_tensor.numel() == 0:
#         print("Empty image found!")

#     # If you want to convert the image tensor to a PIL image for further processing:
#     # Convert the image tensor to a PIL image
#     image_pil = transforms.ToPILImage()(image_tensor)

#     # Check if the PIL image dimensions are zero
#     width, height = image_pil.size
#     if width == 0 or height == 0:
#         print("Empty image found!")

In [ ]:
# Define a filter function that checks if the image has a non-zero shape or pixel values
def filter_empty_images(image, label):
  # Get the shape of the image
  image_shape = tf.shape(image)
  # Check if the shape is non-zero
  non_zero_shape = tf.math.reduce_any(image_shape > 0)
  # Get the sum of the pixel values of the image
  image_sum = tf.math.reduce_sum(image)
  # Check if the sum is non-zero
  non_zero_sum = tf.math.not_equal(image_sum, 0)
  # Return True or False
  return tf.math.logical_and(non_zero_shape, non_zero_sum)

# Apply the filter function to your dataset
train_ds = train_ds.filter(filter_empty_images)


In [25]:
# # Define a custom layer that wraps the resize_with_pad function
# class ResizeWithPadLayer(tf.keras.layers.Layer):
#   def __init__(self, target_height, target_width):
#     super(ResizeWithPadLayer, self).__init__()
#     self.target_height = target_height
#     self.target_width = target_width

#   def call(self, inputs):
#     return tf.image.resize_with_pad(inputs, self.target_height, self.target_width)

# # Create an instance of the custom layer
# resize_with_pad_layer = ResizeWithPadLayer(180, 180)


In [ ]:
# Import the necessary modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the model architecture
model = keras.Sequential([
  # Rescale the pixel values to the range [0, 1]
  layers.Rescaling(1./255, input_shape=(180, 180, 3)),
  # Apply a convolutional layer with 16 filters and a 3x3 kernel
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  # Apply a max pooling layer with a 2x2 window
  layers.MaxPooling2D(),
  # Apply another convolutional layer with 32 filters and a 3x3 kernel
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  # Apply another max pooling layer with a 2x2 window
  layers.MaxPooling2D(),
  # Apply a dropout layer with a rate of 0.2 to reduce overfitting
  layers.Dropout(0.2),
  # Flatten the output of the previous layer
  layers.Flatten(),
  # Apply a dense layer with 128 units and a ReLU activation
  layers.Dense(128, activation='relu'),
  # Apply another dropout layer with a rate of 0.2
  layers.Dropout(0.2),
  # Apply an output layer with the number of classes and a softmax activation
  layers.Dense(2002)
])

# Compile the model with an optimizer, a loss function, and a metric
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
# Train the model for 10 epochs
model.fit(train_ds, epochs=30)

# # Evaluate the model on the test data
# loss, accuracy = model.evaluate(val_ds)

# # Print the loss and accuracy
# print("Loss: ", loss)
# print("Accuracy: ", accuracy)


In [ ]:
model.save('my_model_big_2.h5')
